In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
# import addcopyfighandler

import seaborn as sns

from IPython.core.display import HTML

sys.path.append("../RD-systems-and-test-benches/utils")
import data_processing as dp

import model_fsolve as modf

import scipy.optimize as sco
import scipy.interpolate as sci

import simu as sim

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl

import torch
import torch_model as modto

In [ ]:
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = r"\V4.5_1MPE.xlsx"

In [ ]:
hx, par, cond = fe.initialize(path, file_name)

In [ ]:
list_QF = np.array([50,200,350,500,650])/3600000
list_QF_out = np.array([50,200,350])/3600000
list_alpha = [0.1, 0.4, 0.7, 1]
df = fe.testing_series(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\V4.5_1MPE.xlsx", list_QF, list_QF_out, list_alpha, par, cond)

In [ ]:
df_K = pd.read_excel('G:\Drive partagés\BU04-Innovation\PVT-PL-model\V4.5_1MPE.xlsx')
df_K

In [ ]:
df_A = df_K[['QF', 'QF_out']]
df_A['ratio']=df_A['QF']/df_A['QF_out']
df_A['constant']=1
df_A['alpha']=df_K['alpha']
df_A['QF*alpha'] = df_A['alpha']*df_A['QF']
df_A['QF_out*alpha'] = df_A['alpha']*df_A['QF_out']
df_A['ratio*alpha'] = df_A['alpha']*df_A['ratio']
df_A['alpha2'] = df_A['alpha']**2
df_A['QF*alpha2'] = df_A['alpha2']*df_A['QF']
df_A['QF_out*alpha2'] = df_A['alpha2']*df_A['QF_out']
df_A['ratio*alpha2'] = df_A['alpha2']*df_A['ratio']

In [ ]:
A = df_A.to_numpy()
B = df_K[['Kyin']].to_numpy()
X = np.linalg.lstsq(A,B)[0]

err = (A.dot(X)-B)*100/B
np.abs(err.round()).mean()

In [ ]:
dplt.K_abaque(df2, 'Kyin')


In [ ]:
TUV = pd.read_excel('G:\Drive partagés\BU04-Innovation\Hydraulique\PL_database\\230727_TUV_V4.41_x1_Z_water.xlsx')

In [ ]:
# read excel file
excel_file = pd.ExcelFile(r'G:\Drive partagés\BU04-Innovation\Hydraulique\components\230725_PL_V4.41.xlsx')

# create dictionary with dataframes
df_dict = {}
for sheet_name in excel_file.sheet_names:
    df_dict[sheet_name] = excel_file.parse(sheet_name)
    df_dict[sheet_name].drop(columns='Unnamed: 0',inplace=True)

In [ ]:
for i in range(4):
    key = list(df_dict.keys())[i]
    list_Vdot = np.array(df_dict[key]['Vdot_per_panel'])
    list_Dv = list_Vdot*(i+1)/3600000

    path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
    file_name = f"\V4.41_x{i+1}.xlsx"
    hx, par, cond = fe.initialize(path, file_name)
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv[1::39])

    plt.plot(list_Vdot[1::39], list_PL/1000, label='Simulation')
    plt.plot(list_Vdot, np.array(df_dict[key]['PL']), label='Experiments')
    plt.xlabel('Vdot par panneau [L/h]')
    plt.ylabel('PL [kPa]')
    plt.legend()
    plt.show()



In [ ]:
list(df_dict.keys())

In [ ]:
for k in df_dict.keys():
    print(df_dict[k]['PL'])